In [1]:
# This notebook uses Long Short-Term Memory (LSTM) to predict the closing stock price of a corporation using the past 60 day stock price.

# Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
plt.style.use('fivethirtyeight')

In [2]:
# Get the past year stock price
# today's date
today = pd.to_datetime('today').strftime('%Y-%m-%d')
# 1 year ago
start = pd.to_datetime('today') - pd.DateOffset(years=1)
df = web.DataReader('AAPL', data_source='yahoo', start=start, end=today)

In [3]:
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-09-07,157.259995,154.389999,154.970001,156.690002,82278300.0,155.816879
2021-09-08,157.039993,153.979996,156.979996,155.110001,74420200.0,154.245682
2021-09-09,156.110001,153.949997,155.490005,154.070007,57305700.0,153.211487
2021-09-10,155.479996,148.699997,155.000000,148.970001,140893200.0,148.139893
2021-09-13,151.419998,148.750000,150.630005,149.550003,102404300.0,148.716660
...,...,...,...,...,...,...
2022-08-30,162.559998,157.720001,162.130005,158.910004,77906200.0,158.910004
2022-08-31,160.580002,157.139999,160.309998,157.220001,87991100.0,157.220001
2022-09-01,158.419998,154.669998,156.639999,157.960007,74229900.0,157.960007
